In [ ]:
#packages
%matplotlib inline
import numpy as np 
import pandas as pd 
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split 
import sklearn.metrics as skm
from sklearn.preprocessing import PolynomialFeatures
import operator as op
import sklearn.metrics as skm
import mlxtend as ml
import matplotlib.pyplot as plt 
import seaborn as sns
sns.set(rc={'figure.figsize':(10,8)})
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Data Importing

In [ ]:
# Loading the data
df = pd.read_csv('/kaggle/input/insurance/insurance.csv')
df.head()

# Feature Engineering

In [ ]:
# Enconding categorical variables
df["sex"] = df["sex"].astype('category')
df["sex_cat"] = df["sex"].cat.codes
df["smoker"] = df["smoker"].astype('category')
df["smoker_cat"] = df["smoker"].cat.codes
df["region"] = df["region"].astype('category')
df["region_cat"] = df["region"].cat.codes
df.head()

# Exploratory Data Analysis

In [ ]:
# From this correlation matrix, the strongest predictors
# appears to be smoker_cat, age, and bmi
sns.heatmap(df.corr(), annot = True)

In [ ]:
# Distribution of target variable by smoking status
fig = plt.figure()
gs = fig.add_gridspec(2, 2, hspace=0.4, wspace=0.3)
(ax1, ax2), (ax3, ax4) = gs.subplots(sharex=False, sharey=False)
fig.suptitle('Histograms for Charges')

sns.histplot(ax= ax1, data=df, x ='charges', hue='smoker_cat', 
             bins=10, palette=["green", "red"], stat='count')
ax1.set_title("Charges Count")

sns.histplot(ax= ax2, data=df, x ='charges', hue='smoker_cat', 
             bins=10, palette=["green", "red"], stat='frequency')
ax2.set_title("Charges Freq.")

sns.histplot(ax= ax3, data=df, x ='charges', hue='smoker_cat', 
             bins=10, palette=["green", "red"], stat='count',
            log_scale=True)
ax3.set_title("Log Charges Count")

sns.histplot(ax= ax4, data=df, x ='charges', hue='smoker_cat', 
             bins=10, palette=["green", "red"], stat='frequency',
            log_scale=True)
ax4.set_title("Log Charges Freq.")
plt.show()

In [ ]:
# Scatter_plot btw Charges and BMI
charge_per_bmi = df.eval("charges/bmi").rename("charge_per_bmi")
sns.scatterplot(data=df, x='bmi', y='charges', 
                hue=charge_per_bmi, size=charge_per_bmi)
plt.title('Charges By BMI')

plt.show()

In [ ]:
# Bar Plot
sns.scatterplot(data=df, x='age', y='charges', hue='smoker_cat')

# Model Building & Evaluation

In [ ]:
class reg_models():
    def __init__(self, df):
        self.df = df
        
    def preprocessing(self, target, drop_vars):
        self.X = self.df.drop(drop_vars+[target], axis=1).values
        self.Y = self.df[target].values
        self.X_train, self.X_test, self.Y_train, self.Y_test = train_test_split(self.X, self.Y, 
                                           test_size = 0.2, random_state=0) 
        return self
        
    def fit_predict(self, model_type):
        # Multiple Linear Regression
        if model_type == 'sk-lin':
            model = LinearRegression(normalize=True)
            model.fit(self.X_train, self.Y_train)
            self.Y_pred = model.predict(self.X_test)
            
        if model_type == 'sk-poly':
            reg = LinearRegression(normalize=True)
            pol_feat = PolynomialFeatures(2)
            X_train_transf = pol_feat.fit_transform(self.X_train)
            X_test_transf = pol_feat.fit_transform(self.X_test)

            model = reg.fit(X_train_transf, self.Y_train)
            self.Y_pred = model.predict(X_test_transf)
            
        if model_type == 'rdg':
            model = Ridge(alpha=0.00001, normalize=True)
            model.fit(self.X_train, self.Y_train)
            self.Y_pred = model.predict(self.X_test)
        
        if model_type == 'las':
            model = Lasso(alpha=5, normalize=True)
            model.fit(self.X_train, self.Y_train)
            self.Y_pred = model.predict(self.X_test)
            
        if model_type == 'eln':
            model = ElasticNet(alpha=0.00001, normalize=True)
            model.fit(self.X_train, self.Y_train)
            self.Y_pred = model.predict(self.X_test)

        return self
    
    def performance(self):
        rmse = round(skm.mean_squared_error(self.Y_test, self.Y_pred, squared=False), 2)
        r2_score = round(skm.r2_score(self.Y_test, self.Y_pred), 4)
        ev = round(skm.explained_variance_score(self.Y_test, self.Y_pred), 4)
        mae = round(skm.mean_absolute_error(self.Y_test, self.Y_pred), 2)
    
        return [mae, rmse, r2_score, ev]

# Running
drop_vars = ['sex', 'region', 'smoker']
# Dictionary to store model performance 
models = {}
reg = reg_models(df).preprocessing('charges', drop_vars)
models["Mult. Reg"] = reg.fit_predict('sk-lin').performance()
models["Poly Reg"] = reg.fit_predict('sk-poly').performance()
models["Ridge Reg"] = reg.fit_predict('rdg').performance()
models["Lasso Reg"] = reg.fit_predict('las').performance()
models["ElaNet Reg"] = reg.fit_predict('eln').performance()

#table with the model_performances
models_df = pd.DataFrame.from_dict(models, orient='index',
                  columns=['MAE', 'RSME', 'R_sq', 'Expl. Var.'])
models_df

* Add [interaction terms](https://en.wikipedia.org/wiki/Interaction_(statistics)#In_regression) to model how two or more independent variables together impact the target variable

* Add [spines](http://people.stat.sfu.ca/~cschwarz/Consulting/Trinity/Phase2/TrinityWorkshop/Workshop-handouts/TW-04-Intro-splines.pdf) to approximate piecewise linear models

* Fit [isotonic regression](https://en.wikipedia.org/wiki/Isotonic_regression) to remove any assumption of the target function form

* Fit non-parametric models, such as [MARS](https://en.wikipedia.org/wiki/Multivariate_adaptive_regression_splines)